## Imports

In [7]:
import pandas as pd
import sqlite3

## Load Data

In [4]:
con = sqlite3.connect("../data/nfts.sqlite")
cur = con.cursor()

## List All Available Tables

In [8]:
cur.execute("SELECT * FROM sqlite_master WHERE type='table'")
pd.DataFrame.from_dict(cur.fetchall())

,0,1,2,3,4
0,table,nfts,nfts,2,CREATE TABLE nfts\n (\n address TEXT...
1,table,checkpoint,checkpoint,5,CREATE TABLE checkpoint\n (\n event_...
2,table,mints,mints,6,CREATE TABLE mints\n (\n event_id TE...
3,table,transfers,transfers,8,CREATE TABLE transfers\n (\n event_i...
4,table,transfer_values_quartile_10_distribution_per_a...,transfer_values_quartile_10_distribution_per_a...,10,CREATE TABLE transfer_values_quartile_10_distr...
5,table,current_owners,current_owners,11,CREATE TABLE current_owners(\n nft_address TE...
6,table,current_market_values,current_market_values,12,CREATE TABLE current_market_values(\n nft_add...
7,table,market_values_distribution,market_values_distribution,13,CREATE TABLE market_values_distribution(\n ad...
8,table,transfer_statistics_by_address,transfer_statistics_by_address,14,CREATE TABLE transfer_statistics_by_address(\n...
9,table,transfer_values_quantile_10_distribution_per_a...,transfer_values_quantile_10_distribution_per_a...,15,CREATE TABLE transfer_values_quantile_10_distr...


In [ ]:
con.close()